In [2]:
import numpy as np
from scipy.stats import binom
from scipy.stats import truncnorm
from scipy.stats import gamma

In [3]:
# Parameters 

h = 1/7
pho = 2/3
g = 2/3
R = 19
T = 7*R

In [4]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [5]:
# Mutation transition

def mutation(theta, X):
    #
    X_new = X
    S, E, I, R =  X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3]  
    M = theta[:16].reshape(4,4)
    betas = 1/8000 * np.dot(M, I)
    B = np.random.binomial(S, np.ones(4)-np.exp(-h*betas)) 
    C = np.random.binomial(E, 1-np.exp(-h*pho)) 
    D = np.random.binomial(I, 1-np.exp(-h*g)) 
    S_new = S - B
    E_new = E + B - C
    I_new = I + C - D
    R_new = R + D
    X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3] = S_new, E_new, I_new, R_new
    return X_new

In [6]:
# Likelihood of y knowing theta

def log_f(y, theta, X):
    C = np.zeros((7,4))
    for i in range(7):
        X = mutation(theta, X)
        C[i] = np.random.binomial(X[:, 1], 1-np.exp(-h*pho))
    return(np.log(binom.pmf(y, np.sum(C, axis=0), theta[16:])+1.5))

In [7]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def BootstrapFilter(y, theta, ESS_min, N, R=19):
    X = np.array([[X1, X2, X3, X4] for i in range(N)]) 
    w_init = np.zeros((N, 4))
    X_all = np.array([X for r in range(R+1)])
    #print(X_all)
    for n in range(N):
        w_init[n] = log_f(y[0], theta, X[n])
    W_init = np.zeros((N, 4))
    for n in range(N):
        W_init[n] = np.divide(w_init[n], np.sum(w_init, axis=0))
    W = W_init
    w = w_init
    X_all[1] = X
    #print(X_all)
    l=0
    for r in range(R):
        #print(X)
        w_hats = np.zeros((N,4))
        #print(w)
        for k in range(4):
            #print(k, W[:, k], ESS(W[:, k]))
            if ESS(W[:, k]) < ESS_min:
                A = np.random.choice(np.arange(0, N), size=N, p=W[:, k])
                w_hat = np.zeros(N)
            else:
                l+=1
                A = np.arange(0, N)
                w_hat = w[:, k]
                #print(l)
            for n in range(N):
                X[n][k] = X[A[n]][k]
            w_hats[:, k] = w_hat
        for n in range(N):
            w[n] = w_hats[n]+log_f(y[r], theta, X[n])
        for n in range(N):
            W[n] = np.divide(w[n], np.sum(w, axis=0))
        X_all[r+1] = X
    return(X_all)

In [8]:
theta = np.concatenate((np.random.gamma(5, 1, 16), truncnorm.rvs(a=0, b=1, loc=0.5, scale=0.5, size=4)), axis=None)
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
X_all = BootstrapFilter(Y, theta, 999, 1000, R=19)

In [9]:
print(X_all[8])

[[[  56  246  256  391]
  [  97  457  476  660]
  [  93  827  938 1609]
  [ 103  438  534  820]]

 [[  61  239  288  361]
  [  89  474  438  689]
  [  89  823  941 1614]
  [  93  461  527  814]]

 [[  66  247  262  374]
  [  91  426  455  718]
  [  58  725  920 1764]
  [  84  431  470  910]]

 ...

 [[  52  241  279  377]
  [  90  387  489  724]
  [  64  718  984 1701]
  [  74  444  545  832]]

 [[  46  248  268  387]
  [  75  413  439  763]
  [  70  756  953 1688]
  [  77  467  539  812]]

 [[  58  253  244  394]
  [  75  367  455  793]
  [  66  697  911 1793]
  [  70  440  506  879]]]


In [10]:
def log_prior(theta):
    prior = 1
    for i in range(16):
        prior = prior * gamma.pdf(theta[i], a=5, loc=0, scale=1)
    for i in range(4):
        prior = prior * truncnorm.pdf(theta[16+i], a=0, b=1, loc=0.5, scale=0.5)
    return np.log(1+prior)

In [11]:
def LogL(y, theta, X):
    L = np.zeros(4)
    for r in range(R):
        S = 0
        for n in range(len(X[r])):
            x = X[r][n]
            S = S + np.exp(log_f(y[r], theta, x))
        for k in range(4):
            if S[k]==0:
                L[k] = -100
            else:
                L[k] = L[k] + np.log(S[k]) - len(X[r])
    return L+log_prior(theta)

In [16]:
def PMMH(y, ESS_min, Nx, Nt, R=19):
    theta = np.concatenate((np.random.gamma(5, 1, 16), truncnorm.rvs(a=0, b=1, loc=0.5, scale=0.5, size=4)), axis=None)
    #print(theta)
    thetas = np.array([theta for i in range(Nt)])
    X = BootstrapFilter(y, theta, ESS_min, Nx, R=19)
    L = LogL(y, theta, X)
    for n in range(1, Nt):
        theta_new =  np.concatenate((truncnorm.rvs(a=0, b=np.inf, loc=theta[:16], scale=0.1, size=16), truncnorm.rvs(a=0, b=1, loc=theta[16:], scale=0.1, size=4)), axis=None)
        #print(theta_new)
        X_new = BootstrapFilter(y, theta_new, ESS_min, Nx, R=19)
        L_new = LogL(y, theta_new, X_new)
        v = L_new - L
        for k in range(4):      
            u = np.random.uniform()
            if np.log(u)<v[k]:
                L[k] = L_new[k]
                thetas[n][k:k+4] = theta_new[k:k+4]
                thetas[n][16+k] = theta_new[16+k]
                theta[k:k+4] = theta_new[k:k+4]
                theta[16+k] = theta_new[16+k]
            else:
                thetas[n][k:k+4] = theta[k:k+4]
                thetas[n][16+k] = theta[16+k]    
    return thetas

In [ ]:
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
PMMH(Y, 999, 100, 10000, R=19)